In [5]:
import os


In [6]:
%pwd

'c:\\Users\\91844\\Desktop\\text_summarizer\\Text_Summarization_Project\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\91844\\Desktop\\text_summarizer\\Text_Summarization_Project'

In [9]:
import pandas
from Text_Summarization_Project.logging import logger


In [10]:
# pip uninstall -y transformers torch sentencepiece
# pip install --upgrade pip
# pip install transformers torch sentencepiece


In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    
    root_dir: Path
    data_path: Path 
    tokenizer_name: Path

In [12]:
from Text_Summarization_Project.constants import *
from Text_Summarization_Project.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_file_path: CONFIG_FILE_PATH , params_file_path: PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])   
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config
        

In [14]:
import os
from Text_Summarization_Project.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset ,load_from_disk

In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [16]:
try:
    config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_transformation_config = config_manager.get_data_transformation_config()
    
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert() # fixed the name
    
except Exception as e:
    raise e


[2025-08-14 15:44:34,636 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-08-14 15:44:34,659 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-08-14 15:44:34,662 : INFO : common : created directory at: artifacts]
[2025-08-14 15:44:34,664 : INFO : common : created directory at: artifacts/data_transformation]


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

c:\Users\91844\anaconda3\envs\texts\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]